# 어플리커티브 펑터

In [4]:
:info Functor
:t fmap
-- a를 받아서 b를 주는 함수와 a가 들어있는 펑터를 주면, a에 함수를 적용한 펑터를 반환한다.

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

type Functor :: (* -> *) -> Constraint
class Functor f where
  fmap :: (a -> b) -> f a -> f b
  (<$) :: a -> f b -> f a
  {-# MINIMAL fmap #-}
  	-- Defined in ‘GHC.Base’
instance Functor IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Functor (Either a) -- Defined in ‘Data.Either’
instance Functor ManuallySized -- Defined in ‘ihaskell-diagrams-0.3.2.1:IHaskell.Display.Diagrams.ImgSize’
instance Functor [] -- Defined in ‘GHC.Base’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Functor IO -- Defined in ‘GHC.Base’
instance Functor ((->) r) -- Defined in ‘GHC.Base’
instance Functor ((,,,) a b c) -- Defined in ‘GHC.Base’
instance Functor ((,,) a b) -- Defined in ‘GHC.Base’
instance Functor ((,) a) -- Defined in ‘GHC.Base’
instance Functor ManuallySampled -- Defined in ‘ihaskell-diagrams-0.3.2.1:IHaskell.Display.Diagrams.Animation’

## 펑터값은 추가된 컨텍스트를 갖는 값으로 볼 수 있음

Maybe는 실패한 부가 컨텍스트를 갖는다.

리스트는 한번에 여러 값이 있거나, 아무것도 없을 수도 있음

IO String 또한, IO 컨텍스트를 가지는 String이다. 어떤 문자열을 외부에서 얻어 결과물을 만들어줄 IO작업이라는 의미

In [1]:
main = do line <- fmap reverse getLine  
          putStrLn $ "You said " ++ line ++ " backwards!"  
          putStrLn $ "Yes, you really said" ++ line ++ " backwards!"  
          
main

You said 321 backwards!
Yes, you really said321 backwards!

In [5]:
import qualified Data.Char as Char
import qualified Data.List as List
  
main = do line <- fmap (List.intersperse '-' . reverse . map Char.toUpper) getLine  
          -- intersperse 해당 char로 [string] join 하는 함수
          putStrLn line
          
main

P-A-M-F

## 함수 또한 펑터

(->) r

r -> a 는 (->) r a 로 작성할 수있다.

**-> 는 두개의 타입 인스턴스를 받는 타입 생성자**

(->) r 타입은 매개변수를 하나를 받는 타입생성자가 되므로, 이에 대해서는 Functor instance를 구현할 수 있다

In [7]:
fmap :: (a -> b) -> f a -> f b
fmap :: (a -> b) -> ((->)r a) -> ((->)r b)
-- 이는 (.) 과 같다

Line 2: Redundant bracket
Found:
((->) r a) -> ((->) r b)
Why not:
(->) r a -> ((->) r b)Line 2: Redundant bracket
Found:
((->) r a) -> ((->) r b)
Why not:
((->) r a) -> (->) r b

: 

In [8]:
:t (.)
-- r 대신 a 가 들어가고 b -> c 함수를 받는다.

-- 인자 a를 받는 Functor(Context)는 동일하지만, 안에 있는 반환값(b to c)이 달라진셈 (쉽게 말해, 합성함수)

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

In [15]:
:t fmap (*3) (+100)
fmap (*3) (+100) 1

(*3) `fmap` (+100) $ 1
(*3) . (+100) $ 1
fmap (show . (*3)) (+100) 1

fmap (*3) (+100) :: forall b. Num b => b -> b

303

303

303

"303"

함수 또한 컨텍스트를 가진 값이라는 이야기

컨텍스트를 적용한 것이 함수의 반환 값

**fmap은 두가지로 이해할 수 있다**

- 함수와 펑터 값을 받아 그 펑터 값에 함수를 매핑하는 함수 (Just 3 에 (+3) 해서 Just 6가 됨)
- 함수를 받아서 그 함수를 올리는(lift) 함수, 즉 펑터 값에서 동작 ((+3) 함수에 (*5)를 해서 (\x -> (x + 3) * 5)가 됨)

In [22]:
-- 펑터 값에 함수를 매핑
fmap (replicate 3) [1,2,3,4]
fmap (replicate 3) (Just 4)
fmap (replicate 3) Nothing
fmap (replicate 3) (Left "foo") -- 오류이기 때문에 반복안되고 메시지 그냥 전달
fmap (replicate 3) (Right "blah")

[[1,1,1],[2,2,2],[3,3,3],[4,4,4]]

Just [4,4,4]

Nothing

Left "foo"

Right ["blah","blah","blah"]

## 펑터 규칙

1. 펑터는 매핑될 수 있는 어떤 것처럼 신뢰될수 있게 동작해야함
2. 펑터에서 fmap을 호출하기 위해서는 펑터에 함수를 매핑해야한다.

위는 하스켈에 의해 자동으로 적용되지 않기 때문에, 사용자가 직접 테스트해야한다.

### 규칙1 펑터값에 ID 함수를 매핑하면 돌려받은 펑터값은 원본 펑터값과 동일해야한다

fmap id == id 같아야한다

In [24]:
fmap id (Just 3)
id (Just 3)
fmap id [1..5]
id [1..5]
fmap id []
id []

Line 1: Functor law
Found:
fmap id
Why not:
idLine 2: Redundant id
Found:
id (Just 3)
Why not:
(Just 3)Line 3: Functor law
Found:
fmap id
Why not:
idLine 4: Redundant id
Found:
id [1 .. 5]
Why not:
[1 .. 5]Line 5: Functor law
Found:
fmap id
Why not:
idLine 6: Redundant id
Found:
id []
Why not:
[]

Just 3

Just 3

[1,2,3,4,5]

[1,2,3,4,5]

[]

[]

### 규칙2 두개의 함수를 합친 다음에 합한 함수를 펑터에 매핑하는 것은 펑터에 하나의 함수를 먼저 매핑한 다음에 다른 함수를 매핑하는 것과 동일해야한다.

fmap (f.g) = fmap f . fmap g 와 같다는 이야기이다.

## 규칙에 맞지않는 펑터 만들기

In [27]:
data CMaybe a = CNothing | CJust Int a deriving (Show)

In [29]:
CNothing
CJust 0 "haha"
:t CNothing
:t CJust 0 "haha"
CJust 100 [1,2,3]

CNothing

CJust 0 "haha"

CNothing :: forall a. CMaybe a

CJust 0 "haha" :: CMaybe [Char]

CJust 100 [1,2,3]

In [30]:
instance Functor CMaybe where
    fmap f CNothing = CNothing
    fmap f (CJust counter x) = CJust (counter + 1) (f x)

In [31]:
fmap (++"ha") (CJust 0 "ho")
fmap (++ "he") (fmap (++"ha") (CJust 0 "ho")) -- Int 가 증가하여 id가 맞지 않게됨
fmap (++ "blah") CNothing

Line 2: Functor law
Found:
fmap (++ "he") (fmap (++ "ha") (CJust 0 "ho"))
Why not:
fmap ((++ "he") . (++ "ha")) (CJust 0 "ho")

CJust 1 "hoha"

CJust 2 "hohahe"

CNothing

In [33]:
fmap id (CJust 0 "haha")
id (CJust 0 "haha")
-- 규칙 1에 맞지 않다.
-- 사실 규칙2도 틀림
-- 이런 것을 펑터처럼 사용하게 되면, 오류가 날 수 있다.

Line 1: Functor law
Found:
fmap id
Why not:
idLine 2: Redundant id
Found:
id (CJust 0 "haha")
Why not:
(CJust 0 "haha")

CJust 1 "haha"

CJust 0 "haha"

이러한 규칙을 통해, 펑터가 어떻게 동작할 지에 대해 확신을 가질 수 있게 된다.

## 강화된 펑터, 어플리커티브 펑터

In [35]:
-- 펑터 내의 값에 인자 n개를 받는 함수를 적용하여, 인자 n-1개를 받는 함수가 반환된다

:t fmap (++) (Just "hey")
:t fmap compare (Just 'a')  
:t fmap compare "A LIST OF CHARS"
:t fmap (\x y z -> x + y / z) [3,4,5,6]  

fmap (++) (Just "hey") :: Maybe ([Char] -> [Char])

fmap compare (Just 'a') :: Maybe (Char -> Ordering)

fmap compare "A LIST OF CHARS" :: [Char -> Ordering]

fmap (\x y z -> x + y / z) [3,4,5,6] :: forall a. Fractional a => [a -> a -> a]

펑터값에 "여러 매개변수" 함수들을 매핑하는 방법을 통해 함수를 포함하는 펑터 값들을 얻을 수 있다

In [37]:
a = fmap (*) [1..4]
:t a
fmap (\f -> f 9) a -- 함수가 인자로 전달되었다

a :: forall a. (Num a, Enum a) => [a -> a]

[9,18,27,36]

## Applicative

위 타입 클래스는 두 개의 함수를 가진다

In [40]:
:info Applicative

-- pure와 <*>

type Applicative :: (* -> *) -> Constraint
class Functor f => Applicative f where
  pure :: a -> f a
  (<*>) :: f (a -> b) -> f a -> f b
  GHC.Base.liftA2 :: (a -> b -> c) -> f a -> f b -> f c
  (*>) :: f a -> f b -> f b
  (<*) :: f a -> f b -> f a
  {-# MINIMAL pure, ((<*>) | liftA2) #-}
  	-- Defined in ‘GHC.Base’
instance Applicative (Either e) -- Defined in ‘Data.Either’
instance Applicative IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Applicative [] -- Defined in ‘GHC.Base’
instance Applicative Maybe -- Defined in ‘GHC.Base’
instance Applicative IO -- Defined in ‘GHC.Base’
instance Applicative ((->) r) -- Defined in ‘GHC.Base’
instance (Monoid a, Monoid b, Monoid c) => Applicative ((,,,) a b c) -- Defined in ‘GHC.Base’
instance (Monoid a, Monoid b) => Applicative ((,,) a b) -- Defined in ‘GHC.Base’
instance Monoid a => Applicative ((,) a) -- Defined in ‘GHC.Base’

Applicative 타입 클래스의 어떤 타입 생성자 부분을 만들고자 한다면, 먼저 Functor가 되어야한다.

### `pure :: a -> f a`

값을 받으면 해당 값으로 어플리커티브 값을 반환해야한다. 값을 받아 컨텍스트를 부여한다고 보면됨.

---

### `<*> :: f (a -> b) -> f a  -> f b`

Functor의 `fmap :: (a -> b) -> f a -> f b` 와 비슷하다. fmap이 함수와 펑터 값을 받아서, 펑터 값 안에 함수를 적용했다면, <*>은 함수를 가진 펑터값와 함수의 인자타입의 값을 가진 펑터 값을 받아, 첫번째 인자에서 함수를 추출하여 두번째 펑터 값 안에 적용했다



In [43]:
:info Maybe
-- instance Applicative Maybe -- Defined in ‘GHC.Base’

instance Applicative Maybe where
    pure = Just -- point free
    Nothing <*> _ = Nothing
    (Just f) <*> somthing = fmap f somthing 
    -- 함수를 추출하여 Functor에 정의된 fmap을 활용
    -- 마치 상속된 클래스 같다.
    -- 맞다 사실 typeclass subclass 이다

: 

type Maybe :: * -> *
data Maybe a = Nothing | Just a
  	-- Defined in ‘GHC.Maybe’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Show a => Show (Maybe a) -- Defined in ‘GHC.Show’
instance Eq a => Eq (Maybe a) -- Defined in ‘GHC.Maybe’
instance Semigroup a => Monoid (Maybe a) -- Defined in ‘GHC.Base’
instance Ord a => Ord (Maybe a) -- Defined in ‘GHC.Maybe’
instance Semigroup a => Semigroup (Maybe a) -- Defined in ‘GHC.Base’
instance Read a => Read (Maybe a) -- Defined in ‘GHC.Read’
instance Applicative Maybe -- Defined in ‘GHC.Base’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance MonadFail Maybe -- Defined in ‘Control.Monad.Fail’
instance Traversable Maybe -- Defined in ‘Data.Traversable’

In [48]:
Just (+3) <*> Just 9
pure (+3) <*> Just 10 -- pure로 컨텍스트를 씌운후 <*>로 추출
pure (+3) <*> Just 9
Just (++"hahah") <*> Nothing  
Nothing <*> Just "woot" -- Nothing에서 함수를 추출해와도 아무것도 없기 때문에 Nothing이 계산 값이 됨

Line 2: Use <$>
Found:
pure (+ 3) <*> Just 10
Why not:
(+ 3) <$> Just 10Line 3: Use <$>
Found:
pure (+ 3) <*> Just 9
Why not:
(+ 3) <$> Just 9

Just 12

Just 13

Just 12

Nothing

Nothing

In [50]:
(pure (+) <*> Just 3) <*> Just 5
-- 왼쪽 연관 함수이다

Line 1: Use <$>
Found:
pure (+) <*> Just 3
Why not:
(+) <$> Just 3

Just 8

Applicative functors and the applicative style of doing pure f <*> x <*> y <*> ... allow us to take a function that expects parameters that aren't necessarily wrapped in functors and use that function to operate on several values that are in functor contexts.

어플리커티브 펑터와 `pure f <*> x <*> y <*> ...` 의 어플리커티브 스타일은 어플리커티브 값이 아닌 매개변수를 받는 함수를 받아서 여러 개의 어플리커티브 값들에서 동작하기 위해 그 함수를 사용할 수 있게 해준다.

`pure f <*> x <*> y <*> ...` 대신 `fmap f x <*> y <*> ...`를 사용할 수 있다.

In [56]:
import Control.Applicative
:t  (<$>)
-- fmap 의 중위연산자 버전이다
-- f <$> x = fmap f x

(<$>) :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

In [58]:
-- 세 개의 어플리커티브 값들 간에 함수를 적용하고 싶을 때
-- f <$> x <*> y <*> z
(++) <$> Just "johntra" <*> Just "volta"

Just "johntravolta"

In [59]:
(++) "johntra" "volta"

"johntravolta"

## 리스트도 어플리커티브 펑터

In [61]:
pure "Hey" :: [String]
pure "Hey" :: Maybe String

["Hey"]

Just "Hey"

In [64]:
-- []라는 컨텍스트에 있는 함수를 추출하여 두번째 펑터 값에 적용
-- list comprehension과 같음
[(*0), (+100), (^2)] <*> [1,2,3]
[(+), (*)] <*> [1,2] <*> [3,4]

[0,0,0,101,102,103,1,4,9]

[4,5,5,6,3,4,6,8]

In [65]:
(++) <$> ["ha","heh","hmm"] <*> ["?","!","."]  

["ha?","ha!","ha.","heh?","heh!","heh.","hmm?","hmm!","hmm."]

In [67]:
-- list comprehension과 같음
[ x*y | x <- [2,5,10], y <- [8,10,11]]    
(*) <$> [2,5,10] <*> [8,10,11]  

[16,20,22,40,50,55,80,100,110]

[16,20,22,40,50,55,80,100,110]

In [68]:
filter (>50) $ (*) <$> [2,5,10] <*> [8,10,11]  

[55,80,100,110]

## IO 역시 어플리커티브 펑터

In [ ]:
instance Applicative IO where  
    pure = return  
    a <*> b = do  
        -- do 구문은 여러 IO 동작을 받아 하나로 붙임을 기억하자
        f <- a  -- 함수를 추출
        x <- b  -- 값을 추출
        return (f x)  -- 값에 함수를 적용한 후 컨텍스트 씌움

In [69]:
myAction :: IO String
myAction = do
    a <- getLine
    b <- getLine
    return $ a ++ b

In [70]:
myAction :: IO String
myAction = (++) <$> getLine <*> getLine

## 함수 또한 어플리커티브 펑터

In [ ]:
instance Applicative ((->) r) where  
    -- 인자 타입이 고정된 컨텍스트
    pure x = (\_ -> x)  
    -- pure함수는 해당 값을 돌려주는 컨텍스트
    -- :: a -> r -> a
    f <*> g = \x -> f x (g x)  
    -- f 는 r -> a -> b
    -- g 는 r -> a
    --
    -- :: (->) r (a->b) -> (->) r a -> (->) r b
    -- :: (r -> (a -> b)) -> (r -> a) -> (r -> b)

In [71]:
pure 3 $ "blah"

Line 1: Redundant $
Found:
pure 3 $ "blah"
Why not:
pure 3 "blah"

3

In [74]:
:t (+) <$> (+3) <*> (*100)
(+) <$> (+3) <*> (*100) $ 5
-- \x -> f x (g x)
-- 5 -> f 5 (g 5)
-- 5 -> f 5 (500)
-- 5 -> (5 + 3) + (500)
-- 5 -> 508

(+) <$> (+3) <*> (*100) :: forall b. Num b => b -> b

508

In [78]:
(\x y z -> [x,y,z]) <$> (+3) <*> (*2) <*> (/2) $ 5


f = (\x y z -> [x,y,z]) <$> (+3)
-- f = (\x -> \y z -> [x,y,z]) <$> (+3)
-- f <$> g :: (a->b) -> f a -> f b
-- \a -> \y z -> [(a+3), y, z]
:t f

f' = f <*> (*2) 
-- f <*> g :: (r -> a -> b) -> (r -> a) -> (r -> b)
-- \b -> f b (b*2)
-- \b -> \z -> [(b+3), (b*2), z]
:t f'


f'' = f' <*> (/2)
-- \c -> f' c (c/2)
-- \c -> [(c+3), (c*2), (c/2)]
:t f''

f'' 5

[8.0,10.0,2.5]

f :: forall a. Num a => a -> a -> a -> [a]

f' :: forall a. Num a => a -> a -> [a]

f'' :: forall a. Fractional a => a -> [a]

[8.0,10.0,2.5]

In [79]:
[(+3), (*2)] <*> [1,2]

[4,5,2,4]

## ZipList

In [ ]:
instance Applicative ZipList where  
-- ZipList는 생성자이다
-- List가 두개의 타입인스턴스를 가질수 없으므로 별도로 생성됨
        pure x = ZipList (repeat x)  
        ZipList fs <*> ZipList xs = ZipList (zipWith (\f x -> f x) fs xs)
        
        
        -- f<*>g :: f (a -> b) -> f a -> f b

In [83]:
getZipList $ (+) <$> ZipList [1,2,3] <*> ZipList [100,100,100]
getZipList $ (+) <$> ZipList [1,2,3] <*> ZipList [100,100..]
getZipList $ (,,) <$> ZipList "dog" <*> ZipList "cat" <*> ZipList "rat"

-- (,,) 는 \x y z -> (x,y,z) 와 같다

[101,102,103]

[101,102,103]

[('d','c','r'),('o','a','a'),('g','t','t')]

## 어플리커티브 규칙

0. `pure f <*> x = fmap f x` 함수에 컨텍스트를 입혔다가, 벗겨서 적용해도 똑같은것
1. `pure id <*> v = v` : 어플리커티브 방식으로 id를 적용해도 값이 같을것
2. `pure (.) <*> u <*> v <*> w = u <*> (v <*> w)` : 순서가 바뀌어도 됨
3. `pure f <*> pure x = pure (f x)` : 합성이 된다
4. `u <*> pure y = pure ($y) <*> u`

## 어플리커티브를 위한 유용한 함수들

In [85]:
:t liftA2
-- liftA2 f a b = f <$> a <*> b

liftA2 :: forall (f :: * -> *) a b c. Applicative f => (a -> b -> c) -> f a -> f b -> f c

어플리커티브 펑터는 여러개의 펑터 값들 간에 하나의 함수를 매핑할 수 있다.
Just 4 Just 9 간에 (+) 를 적용핧 수 있다... 정도?

In [87]:
fmap (\x -> [x]) (Just 4)

Line 1: Use :
Found:
\ x -> [x]
Why not:
(: [])

Just [4]

In [89]:
liftA2 (:) (Just 3) (Just [4])
(:) <$> Just 3 <*> Just [4]

Just [3,4]

Just [3,4]

In [3]:
sequenceA :: (Applicative f) => [f a] -> f [a]
sequenceA [] = pure []
sequenceA (x:xs) = (:) <$> x <*> sequenceA xs
-- x가 현재 f a 타입이다
-- f [x:x:x: ...] 와 같이 재귀됨

Line 2: Use foldr
Found:
sequenceA [] = pure []
sequenceA (x : xs) = (:) <$> x <*> sequenceA xs
Why not:
sequenceA xs = foldr (\ x -> (<*>) ((:) <$> x)) (pure []) xs

In [99]:
sequenceA [Just 1, Just 2]
(:) <$> Just 1 <*> sequenceA [Just 2]

Just [1,2]

Just [1,2]

In [2]:
sequenceA :: (Applicative f) => [f a] -> f [a]
sequenceA = foldr (liftA2 (:)) (pure [])
-- f and acc

: 

In [ ]:
sequenceA [Just 3, Just 2, Just 1]
sequenceA [Just 3, Nothing, Just 1]
sequenceA [(+3),(+2),(+1)] 3 -- [(->) Int Int] 가 (->) Int [int] 가 되면서 Int를 받는 함수가 됨
sequenceA [[1,2,3], [4,5,6]] 
-- [[Int]] 가 [[Int]]로 변함 (안쪽 []가 바깥쪽 [] 와 위치 변경)
-- (:) <$> [1,2,3] <*> sequenceA [[4,5,6]]
-- [1:, 2:, 3:] <*> ( (:) <$> [4,5,6] <*> sequenceA [] )
-- [1:, 2:, 3:] <*> ( (:) <$> [4,5,6] <*> [] )
-- [1:, 2:, 3:] <*> ( [4:,5:,6:] <*> [] )
-- [1:, 2:, 3:] <*> [4:,5:,6:]
-- list comprehension!
-- [[1,4], [1,5], [1,6], [2,4], [2,5], [2,6], [3,4], [3,5], [3,6]]




Line 17: Redundant bracket
Found:
(:) <$> [] <*> (sequenceA [])
Why not:
(:) <$> [] <*> sequenceA []

Just [3,2,1]

Nothing

[6,5,4]

[[1,4],[1,5],[1,6],[2,4],[2,5],[2,6],[3,4],[3,5],[3,6]]

[]

In [5]:
sequenceA [[1,2,3],[4,5,6],[3,4,4],[]]  

-- empty list와의 list comprehension에서 줄줄이 empty list로 변하는 것
(:) <$> [1,2,3] <*> ( (:) <$> [4,5,6] <*> ( (:) <$> [3,4,4] <*> ( (:) <$> [] <*> (sequenceA []) ) ) )
(:) <$> [1,2,3] <*> ( (:) <$> [4,5,6] <*> ( (:) <$> [3,4,4] <*> ( (:) <$> [] <*> [] ) ) )
(:) <$> [1,2,3] <*> ( (:) <$> [4,5,6] <*> ( (:) <$> [3,4,4] <*> ( [] <*> [] ) ) )
(:) <$> [1,2,3] <*> ( (:) <$> [4,5,6] <*> ( (:) <$> [3,4,4] <*> [] ) )
(:) <$> [1,2,3] <*> ( (:) <$> [4,5,6] <*> ( [(:)3,(:)4,(:)4] <*> [] ) )
(:) <$> [1,2,3] <*> ( (:) <$> [4,5,6] <*> [] )
(:) <$> [1,2,3] <*> ( [(:)4,(:)5,(:)6] <*> [] )
(:) <$> [1,2,3] <*> []
[(:)1,(:)2,(:)3] <*> []
[]

Line 4: Redundant bracket
Found:
(:) <$> [] <*> (sequenceA [])
Why not:
(:) <$> [] <*> sequenceA []

[]

[]

[]

[]

...

[]

[]

[]

[]

[]

In [13]:
map (\f -> f 7) [(>4), (<10), odd]
and $ map (\f -> f 7) [(>4), (<10), odd]

Line 2: Use all
Found:
and $ map (\ f -> f 7) [(> 4), (< 10), odd]
Why not:
all (\ f -> f 7) [(> 4), (< 10), odd]

[True,True,True]

True

In [8]:
sequenceA [(>4), (<10), odd] 7
-- [(->) Int Bool] ->  (->) Int [Bool]
(:) <$> (>4) <*> ((:) <$> (<10) <*> ((:) <$> odd <*> (sequenceA []))) $ 7
-- (:) <$> (>4) <*> ((:) <$> (<10) <*> ((:) <$> odd <*> []) $ 7
(:) <$> (>4) <*> ((:) <$> (<10) <*> ((:) <$> odd <*> (pure []))) $ 7
-- pure [] :: Int -> [Bool]
-- (:) . odd :: Int -> [Bool]
-- f <*> g :: f (a -> b) -> f a -> f b
-- f <*> g :: Int -> ([Bool] -> [Bool]) -> (Int -> [Bool]) -> (Int -> [Bool])
-- \x -> f x (g x)
-- \x: Int ->  f x:int [Bool]
:t ((:) <$> odd)

[True,True,True]

[True,True,True]

[True,True,True]

((:) <$> odd) :: forall a. Integral a => a -> [Bool] -> [Bool]

In [21]:
and $ sequenceA [(>4),(<10),odd] 7  

True

In [26]:
sequenceA [[1,2,3],[4,5,6]]
[[x,y] | x <- [1,2,3], y <- [4,5,6]]
sequenceA [[1,2],[3,4]]
[[x,y] | x <- [1,2], y <- [3,4]]  
sequenceA [[1,2],[3,4],[5,6]]  
[[x,y,z] | x <- [1,2], y <- [3,4], z <- [5,6]]  

[[1,4],[1,5],[1,6],[2,4],[2,5],[2,6],[3,4],[3,5],[3,6]]

[[1,4],[1,5],[1,6],[2,4],[2,5],[2,6],[3,4],[3,5],[3,6]]

[[1,3],[1,4],[2,3],[2,4]]

[[1,3],[1,4],[2,3],[2,4]]

[[1,3,5],[1,3,6],[1,4,5],[1,4,6],[2,3,5],[2,3,6],[2,4,5],[2,4,6]]

[[1,3,5],[1,3,6],[1,4,5],[1,4,6],[2,3,5],[2,3,6],[2,4,5],[2,4,6]]

In [4]:
sequenceA [getLine, getLine, getLine]

-- sequenceA :: [f a] -> f [a]
-- sequenceA :: [IO String] -> IO [String]

-- sequenceA [] = []
-- sequenceA (x:xs) = (:) <$> x <*> sequenceA xs
-- (:) <$> getLine <*> ((:) <$> getLine <*> ((:) <$> getLine <*> []))
-- 

["1","2","3"]

In [3]:
:t <*>

: 

In [ ]:
IO (String -> [String]) <*> []
f <*> g :: f (a -> b) -> (f a) -> (f b)

